# KATL Final Approach Simulation
### ! conda activate tracon

In [1]:
import numpy as np
import pandas as pd
import math

# ==========================================
# 1. PARAMETERS & CONSTANTS
# ==========================================

# Simulation Constraints
R_TURN_NM = 6.0       # Radius of RF turn (nmi)
T_SEP_SEC = 66.0       # Minimum separation (seconds)
T_MAX_SEC = 3600.0     # Simulation duration (1 hour)

# Unit Conversions
KM_TO_NM = 0.539957    # 1 Kilometer = 0.539957 Nautical Miles
R_EARTH_KM = 6371.0    # Radius of Earth in KM

# Reference Point: KATL Runway 9L Threshold (Origin 0,0)
REF_POINT = {'lat': 33.634667, 'lon': -84.448000}

# Fix Definitions (Lat/Lon)
# Note: VINII is included here to calculate its relative X/Y position
raw_fixes = {
    'VINII': {'lat': 33.634650, 'lon': -84.549842, 'type': 'FAF', 'flow': 'FAF'},
    'DALAS': {'lat': 33.952250, 'lon': -84.848022, 'type': 'Corner', 'flow': 'NorthWest'},
    'LOGEN': {'lat': 33.988050, 'lon': -84.056786, 'type': 'Corner', 'flow': 'NorthEast'},
    'HUSKY': {'lat': 33.330458, 'lon': -83.980208, 'type': 'Corner', 'flow': 'SouthEast'},
    'TIROE': {'lat': 33.306453, 'lon': -84.866031, 'type': 'Corner', 'flow': 'SouthWest'}
}

# ==========================================
# 2. GEOMETRY FUNCTIONS (Great Circle)
# ==========================================

def get_distance_and_bearing(lat1, lon1, lat2, lon2):
    """
    Calculates Great Circle distance (km) and initial bearing (degrees)
    between two points.
    """
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    # --- Haversine Formula for Distance ---
    a = math.sin(dphi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    dist_km = R_EARTH_KM * c
    
    # --- Bearing Formula ---
    y = math.sin(dlambda) * math.cos(phi2)
    x = math.cos(phi1) * math.sin(phi2) - math.sin(phi1) * math.cos(phi2) * math.cos(dlambda)
    bearing_rad = math.atan2(y, x)
    bearing_deg = (math.degrees(bearing_rad) + 360) % 360
    
    return dist_km, bearing_deg

def geo_to_cartesian(target_lat, target_lon, ref_lat, ref_lon):
    """
    Converts target Lat/Lon to X (East) / Y (North) in Nautical Miles
    relative to the reference point (0,0).
    """
    # 1. Get Distance (km) and Bearing (deg)
    dist_km, bearing_deg = get_distance_and_bearing(ref_lat, ref_lon, target_lat, target_lon)
    
    # 2. Convert to NM
    dist_nm = dist_km * KM_TO_NM
    
    # 3. Project to Cartesian (Standard Math: 0 deg = East, but Map: 0 deg = North)
    # Map Bearing: 0 is North (Y+), 90 is East (X+)
    bearing_rad = math.radians(bearing_deg)
    
    x_nm = dist_nm * math.sin(bearing_rad) # Sin gives East component for Map Bearing
    y_nm = dist_nm * math.cos(bearing_rad) # Cos gives North component for Map Bearing
    
    return x_nm, y_nm

# ==========================================
# 3. INITIALIZE GEOMETRY
# ==========================================

processed_fixes = {}

for name, data in raw_fixes.items():
    x, y = geo_to_cartesian(data['lat'], data['lon'], REF_POINT['lat'], REF_POINT['lon'])
    
    processed_fixes[name] = data.copy()
    processed_fixes[name]['x'] = x
    processed_fixes[name]['y'] = y
    processed_fixes[name]['dist (nmi)'] = np.sqrt(x**2 + y**2)
    processed_fixes[name]['flow'] = data['flow']

print("Geometry Setup Complete.")
print(f"Origin (Runway 9L Threshold): (0.0, 0.0) nm")
print(f"FAF (VINII) Calculated Location: ({processed_fixes['VINII']['x']:.4f}, {processed_fixes['VINII']['y']:.4f}) nm")
pd.DataFrame(processed_fixes).T[['type', 'x', 'y', 'dist (nmi)', 'flow']] # Display Table

Geometry Setup Complete.
Origin (Runway 9L Threshold): (0.0, 0.0) nm
FAF (VINII) Calculated Location: (-5.0910, 0.0015) nm


,type,x,y,dist (nmi),flow
VINII,FAF,-5.090967,0.001485,5.090968,FAF
DALAS,Corner,-19.922658,19.106464,27.603791,NorthWest
LOGEN,Corner,19.475811,21.254227,28.827927,NorthEast
HUSKY,Corner,23.466697,-18.211933,29.704551,SouthEast
TIROE,Corner,-20.976252,-19.663862,28.751881,SouthWest


In [2]:
# ==========================================
# 4. ARRIVAL GENERATION FUNCTIONS
# ==========================================

def generate_shifted_poisson_times(lambda_ph, t_sep, t_max):
    """
    Generates a list of arrival times (seconds) based on Shifted Poisson Process.
    Delta_t = T_sep + Exp(1/Lambda)
    """
    lambda_per_sec = lambda_ph / 3600.0
    arrivals = []
    t = 0.0
    
    while t < t_max:
        # Stochastic component: Exponential distribution
        stochastic_wait = np.random.exponential(1.0 / lambda_per_sec)
        
        # Total interval: Hard separation + Stochastic
        inter_arrival = t_sep + stochastic_wait
        
        t += inter_arrival
        if t > t_max:
            break
        arrivals.append(t)
        
    return arrivals

def generate_scenario(seed=None):
    np.random.seed(seed)
    all_flights = []
    
    # Retrieve FAF location for Logic checks
    faf_x = processed_fixes['VINII']['x']
    faf_y = processed_fixes['VINII']['y']

    # Iterate only through the 4 corners (exclude VINII from being a start point)
    corners = [k for k, v in processed_fixes.items() if v['type'] == 'Corner']
    
    print(f"{'Corner':<10} | {'Lambda (arr/hr)':<15} | {'Count'}")
    print("-" * 40)

    for name in corners:
        fix_data = processed_fixes[name]
        
        # 1. Sample Demand (Log-Uniform between 5 and 45 aircraft/hr)
        # This ensures we test both low and high traffic scenarios
        log_lambda = np.random.uniform(np.log(5), np.log(25))
        lambda_s = np.exp(log_lambda)
        
        # 2. Generate Times
        times = generate_shifted_poisson_times(lambda_s, T_SEP_SEC, T_MAX_SEC)
        
        # 3. Determine Geometry Logic
        # North/South Logic (for turn direction center calculation later)
        is_north = fix_data['y'] > faf_y
        
        # LongArc Logic: 
        # If Fix X > FAF X, plane is "downwind" relative to FAF -> LongArc
        is_long_arc = 1 if fix_data['x'] > faf_x else 0
        
        print(f"{name:<10} | {lambda_s:>15.2f} | {len(times)}")
        
        # 4. Store Data
        for t in times:
            all_flights.append({
                'entry_time': t,
                'corner': fix_data['flow'],
                'enter_fix_name': name,
                # Entry State
                'x_entry': fix_data['x'],
                'y_entry': fix_data['y'],
                # Target State (FAF)
                'x_faf': faf_x,
                'y_faf': faf_y,
                # Optimization Flags
                'r_turn': R_TURN_NM,
                'is_north': is_north,
                'long_arc': is_long_arc
            })
            
    # Create DataFrame and sort by time
    df = pd.DataFrame(all_flights).sort_values('entry_time').reset_index(drop=True)
    df['aircraft_id'] = df.index + 1
    
    return df

# ==========================================
# 5. EXECUTE GENERATION
# ==========================================

df_arrivals = generate_scenario(seed=None)

print("\nSimulation Generated.")
print(f"Total Aircraft: {len(df_arrivals)}")
print("\nFirst 10 Arrivals:")
display(df_arrivals.head(10))

Corner     | Lambda (arr/hr) | Count
----------------------------------------
DALAS      |           24.43 | 15
LOGEN      |            9.43 | 9
HUSKY      |           15.87 | 12
TIROE      |           11.46 | 9

Simulation Generated.
Total Aircraft: 45

First 10 Arrivals:


,entry_time,corner,enter_fix_name,x_entry,y_entry,x_faf,y_faf,r_turn,is_north,long_arc,aircraft_id
0,84.295182,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,1
1,166.813758,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,2
2,168.418253,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,3
3,258.758770,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,4
4,389.716874,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,5
5,391.302385,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,6
6,477.332348,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,7
7,622.363538,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,8
8,732.205749,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,9
9,743.612044,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,10


# Pyomo Optimization

# Setup I: Optimize $d_i$ Only with FCFS (Fixed Sequence)

In [3]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# ==========================================
# 6. OPTIMIZATION MODEL (Fixed Sequence)
# ==========================================

def solve_schedule_optimization(df_arrivals, debug=False):
    """
    Builds and solves the NLP model to find optimal d_i (extension)
    to minimize makespan while satisfying T_sep.
    """
    
    # --- A. Setup Model ---
    m = pyo.ConcreteModel()
    
    # Indices (Aircraft IDs)
    # We assume the dataframe is already sorted by entry time (FCFS)
    flight_ids = df_arrivals['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    # Parameters (Fixed Data from DF)
    # We use dictionaries to map ID -> Value for Pyomo
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x_entry = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y_entry = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_is_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long_arc = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    # Global Geometry Constants
    r = R_TURN_NM
    x_faf = processed_fixes['VINII']['x']
    y_faf = processed_fixes['VINII']['y']
    
    # Speeds (Knots) - Fixed for this formulation
    # v_L: Tangent leg, v_theta: Turn, v_f: Final straight
    v_L = 210.0
    v_theta = 180.0
    v_f = 150.0
    
    # Bounds for d_i (The extension leg)
    # Min 0.0, Max 20.0 nm (approx 37km extension limit)
    d_min, d_max = 0.0, 20.0 
    
    # --- B. Decision Variables ---
    # d[i]: The extension distance from FAF (nautical miles)
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(d_min, d_max), initialize=0.0)
    
    # t[i]: The arrival time at FAF (seconds)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=lambda m, i: p_tau[i] + 500)

    # --- C. Geometry Expressions (The Math) ---
    # We define these as expressions so Pyomo can calculate derivatives
    
    def calc_travel_time_expr(m, i):
        # 1. Determine Center of Turn C0 based on d[i]
        # If North Arrival: Center is (X_faf - d, Y_faf + r)
        # If South Arrival: Center is (X_faf - d, Y_faf - r)
        # We use standard python if because 'p_is_north' is a parameter (constant)
        
        y_center_offset = r if p_is_north[i] else -r
        
        # Coordinates of Center C0
        Cx = x_faf - m.d[i]
        Cy = y_faf + y_center_offset
        
        # Coordinates of Projected Center C0_prime
        Cx_prime = x_faf - m.d[i]
        Cy_prime = y_faf
        
        # 2. Calculate Distances d0 and d0_prime
        # Distance from Entry (Px, Py) to Center C0
        # pyo.sqrt is required for variables
        d0_sq = (p_x_entry[i] - Cx)**2 + (p_y_entry[i] - Cy)**2
        d0 = pyo.sqrt(d0_sq)
        
        # Distance from Entry to Projected Center C0_prime
        d0_prime_sq = (p_x_entry[i] - Cx_prime)**2 + (p_y_entry[i] - Cy_prime)**2
        
        # 3. Tangent Distance d_L (Pythagoras)
        # d_L = sqrt(d0^2 - r^2)
        # Small epsilon 1e-6 added to prevent sqrt(0) error if d0=r
        d_L = pyo.sqrt(d0_sq - r**2 + 1e-6)
        
        # 4. Angle Calculation (Thetas)
        # theta1 = acos((r^2 + d0^2 - d_L^2) / (2*r*d0)) -> Simplifies to acos(r/d0)
        # theta2 = acos((r^2 + d0^2 - d0_prime^2) / (2*r*d0))
        
        # We clamp inputs to acos to [-1, 1] to prevent numerical errors
        term1 = r / d0
        term2 = (r**2 + d0_sq - d0_prime_sq) / (2 * r * d0)
        
        theta1 = pyo.acos(term1)
        theta2 = pyo.acos(term2)
        
        # 5. Total Turn Angle theta
        if p_long_arc[i]:
            # Downwind: 2*pi - (t1 + t2)
            theta_radians = 2*3.14159 - (theta1 + theta2)
        else:
            # Base/Straight: t2 - t1
            theta_radians = theta2 - theta1
            
        # 6. Distances
        dist_turn = r * theta_radians
        dist_final = m.d[i] # The straight-in extension
        
        # 7. Total Time (in Seconds)
        # Distance (nm) / Speed (knots) * 3600 (sec/hr)
        time_L = (d_L / v_L) * 3600
        time_turn = (dist_turn / v_theta) * 3600
        time_final = (dist_final / v_f) * 3600
        
        return p_tau[i] + time_L + time_turn + time_final

    # --- D. Constraints ---
    
    # 1. Link Physics to Time Variable
    def physics_rule(m, i):
        return m.t[i] == calc_travel_time_expr(m, i)
    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)
    
    # 2. Separation Constraint (Sequence Preserved)
    def separation_rule(m, i):
        # Skip the first aircraft
        prev_i = i - 1
        if prev_i not in m.I:
            return pyo.Constraint.Skip
        return m.t[i] >= m.t[prev_i] + T_SEP_SEC
    m.c_separation = pyo.Constraint(m.I, rule=separation_rule)
    
    # --- E. Objective ---
    # Minimize the arrival time of the LAST aircraft (Makespan)
    last_id = flight_ids[-1]
    m.obj = pyo.Objective(expr=m.t[last_id], sense=pyo.minimize)
    
    # --- F. Solve ---
    solver = SolverFactory('ipopt')
    solver.options['max_iter'] = 10000
    if not debug:
        solver.options['print_level'] = 0 # Suppress solver text
        
    result = solver.solve(m, tee=debug)
    
    # --- G. Extract Results ---
    results = []
    for i in m.I:
        results.append({
            'aircraft_id': i,
            'optimized_d_i': pyo.value(m.d[i]),
            'arrival_time_faf': pyo.value(m.t[i]),
            'scheduled_entry': p_tau[i]
        })
        
    return pd.DataFrame(results)

# ==========================================
# 7. RUN OPTIMIZATION
# ==========================================

print("Optimizing Schedule...")
df_opt_results = solve_schedule_optimization(df_arrivals, debug=True)

# Merge results back with original info for viewing
df_final = pd.merge(df_arrivals, df_opt_results, on='aircraft_id')

# Calculate Delay (Arrival Time - Minimum Possible Time)
# Note: This is simplified; true delay is typically (Actual - Scheduled)
# Here we just look at the timeline.
df_final['transit_time'] = df_final['arrival_time_faf'] - df_final['entry_time']

print("\nOptimization Complete.")
display(df_final[['aircraft_id', 'corner', 'entry_time', 'optimized_d_i', 'arrival_time_faf', 'transit_time']])

Optimizing Schedule...
Ipopt 3.14.19: max_iter=10000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       90
Number of nonzeros in inequality constraint Jacobian.:       88
Number of nonzeros in Lagrangian Hessian.............:       45

Total number of variables............................:       90
                     variables with only lower bounds:       45
                variables with lower and upper bounds:       45
                     variables with only upper bounds:        0
Total number of equality constraints..

,aircraft_id,corner,entry_time,optimized_d_i,arrival_time_faf,transit_time
0,1,SouthWest,84.295182,7.187966e+00,680.754054,596.458872
1,2,SouthEast,166.813758,-9.999978e-09,1023.177678,856.363920
2,3,NorthWest,168.418253,2.000000e+01,1067.919962,899.501710
3,4,NorthWest,258.758770,1.929762e+01,1138.669672,879.910902
4,5,SouthWest,389.716874,1.541875e+01,1209.307024,819.590150
5,6,NorthWest,391.302385,1.971112e+01,1279.844180,888.541795
6,7,SouthWest,477.332348,1.886226e+01,1362.562782,885.230433
7,8,SouthEast,622.363538,-9.999978e-09,1488.162497,865.798959
8,9,NorthWest,732.205749,1.557521e+01,1544.807765,812.602017
9,10,SouthWest,743.612044,1.734763e+01,1601.834968,858.222924


## Introducing Slack Variables for High Density Scenarios

In [4]:
# ==========================================
# 6. OPTIMIZATION MODEL (Fixed Speed + Slack)
# ==========================================

def solve_schedule_optimization_fixed_speed_robust(df_arrivals, debug=False):
    """
    Implements Section 2.2: Optimize d_i only.
    Includes Slack Variables to handle infeasibility during high traffic.
    """
    
    m = pyo.ConcreteModel()
    
    # --- A. Sets & Parameters ---
    flight_ids = df_arrivals['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    # Dictionaries for fast lookup
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x_entry = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y_entry = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_is_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long_arc = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    # Geometry Constants
    r = R_TURN_NM
    x_faf = processed_fixes['VINII']['x']
    y_faf = processed_fixes['VINII']['y']
    
    # FIXED SPEEDS (Knots) - Constants, not Variables
    v_L_const = 210.0
    v_theta_const = 180.0
    v_f_const = 150.0
    
    # --- B. Decision Variables ---
    
    # 1. Path Extension (The only control variable)
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    
    # 2. Arrival Time (Dependent variable)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=lambda m, i: p_tau[i] + 500)

    # 3. Slack Variable (The "Safety Valve")
    # Represents seconds of violation allowed to prevent solver crash
    m.slack = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=0.0)

    # --- C. Geometry Expressions ---
    def calc_travel_time_expr(m, i):
        # Center Calculation
        y_center_offset = r if p_is_north[i] else -r
        Cx = x_faf - m.d[i]
        Cy = y_faf + y_center_offset
        Cx_prime = x_faf - m.d[i]
        Cy_prime = y_faf
        
        # Distances (d0, d0_prime)
        d0_sq = (p_x_entry[i] - Cx)**2 + (p_y_entry[i] - Cy)**2
        d0 = pyo.sqrt(d0_sq + 1e-6)
        d0_prime_sq = (p_x_entry[i] - Cx_prime)**2 + (p_y_entry[i] - Cy_prime)**2
        
        # Tangent Leg (d_L)
        d_L = pyo.sqrt(d0_sq - r**2 + 1e-6)
        
        # Angles
        term1 = r / (d0 + 1e-6)
        term2 = (r**2 + d0_sq - d0_prime_sq) / (2 * r * (d0 + 1e-6))
        theta1 = pyo.acos(term1)
        theta2 = pyo.acos(term2)
        
        if p_long_arc[i]:
            theta_rad = 2*3.14159 - (theta1 + theta2)
        else:
            theta_rad = theta2 - theta1
            
        # Distances
        dist_turn = r * theta_rad
        dist_final = m.d[i]
        
        # Time Calculation using FIXED SPEEDS
        # (nm / knots) * 3600 = seconds
        t_L = (d_L / v_L_const) * 3600
        t_turn = (dist_turn / v_theta_const) * 3600
        t_final = (dist_final / v_f_const) * 3600
        
        return p_tau[i] + t_L + t_turn + t_final

    # --- D. Constraints ---
    
    # 1. Physics Constraint
    def physics_rule(m, i):
        return m.t[i] == calc_travel_time_expr(m, i)
    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)
    
    # 2. Robust Separation Constraint
    def separation_rule(m, i):
        prev_i = i - 1
        if prev_i not in m.I:
            return pyo.Constraint.Skip
        
        # Constraint: Time[i] >= Time[i-1] + 64 - Slack[i]
        return m.t[i] >= m.t[prev_i] + T_SEP_SEC - m.slack[i]
    m.c_separation = pyo.Constraint(m.I, rule=separation_rule)
    
    # --- E. Objective ---
    # Minimize Last Arrival Time + Penalty for Violations
    # Weight = 1000. This means the solver will do EVERYTHING possible to extend d_i
    # before it dares to use 1 second of slack.
    last_id = flight_ids[-1]
    m.obj = pyo.Objective(expr=m.t[last_id] + 1000 * sum(m.slack[i] for i in m.I), sense=pyo.minimize)
    
    # --- F. Solve ---
    solver = SolverFactory('ipopt')
    if not debug: solver.options['print_level'] = 0
    
    # Ipopt options for robustness
    solver.options['max_iter'] = 10000
    solver.options['tol'] = 1e-6
    
    solver.solve(m, tee=debug)
    
    # --- G. Extract Results ---
    results = []
    for i in m.I:
        results.append({
            'aircraft_id': i,
            'entry_time': p_tau[i],
            'optimized_d_i': pyo.value(m.d[i]),
            'arrival_time_faf': pyo.value(m.t[i]),
            'actual_separation': T_SEP_SEC-pyo.value(m.slack[i]),
            'scheduled_delay': pyo.value(m.t[i]) - p_tau[i]
        })
        
    return pd.DataFrame(results)

# --- Run Optimization ---
print("Running Robust Optimization (Fixed Speed, d_i only)...")
df_results = solve_schedule_optimization_fixed_speed_robust(df_arrivals, debug=True)

# Merge and Display
df_final = pd.merge(df_arrivals, df_results[['aircraft_id', 'optimized_d_i', 'arrival_time_faf', 'actual_separation']], on='aircraft_id')


# Check total violation
if df_final['actual_separation'].any() < T_SEP_SEC:
    print("WARNING: Traffic demand exceeded capacity. Some aircraft were not separated by 64s.")
else:
    print("SUCCESS: All aircraft separated by 64s using only path extension.")

df_final

Running Robust Optimization (Fixed Speed, d_i only)...


Ipopt 3.14.19: max_iter=10000
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       90
Number of nonzeros in inequality constraint Jacobian.:      132
Number of nonzeros in Lagrangian Hessian.............:       45

Total number of variables............................:      135
                     variables with only lower bounds:       90
                variables with lower and upper bounds:       45
                     variables with only upper bounds:        0
Total number of equality constraints...............

,entry_time,corner,enter_fix_name,x_entry,y_entry,x_faf,y_faf,r_turn,is_north,long_arc,aircraft_id,optimized_d_i,arrival_time_faf,actual_separation
0,84.295182,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,1,7.744809e+00,693.144912,66.000000
1,166.813758,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,2,-9.977764e-09,1041.059402,66.000000
2,168.418253,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,3,2.000000e+01,1063.129688,22.070287
3,258.758770,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,4,1.905152e+01,1133.645048,66.000000
4,389.716874,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,5,1.529835e+01,1205.430087,66.000000
5,391.302385,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,6,1.959249e+01,1277.343958,66.000000
6,477.332348,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,7,1.879973e+01,1361.390405,66.000000
7,622.363538,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,8,-9.977764e-09,1496.609181,66.000000
8,732.205749,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,9,1.597395e+01,1551.162595,54.553415
9,743.612044,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,10,1.764894e+01,1606.980376,55.817782


# Setup II: Co-optimize $d_i$ and segments speed 

In [5]:
# ==========================================
# 7. CO-OPTIMIZATION MODEL (Hard Constraints)
#    Variables: d_i, v_L, v_theta, v_f
#    Constraint: STRICT 64s separation
# ==========================================

def solve_co_optimization_no_slack(df_arrivals, debug=False):
    """
    Implements Section 2.3: Co-optimize d_i and Speeds.
    NO Slack variables. Enforces strict T_sep.
    May result in 'Infeasible' if traffic demand is too high.
    """
    
    m = pyo.ConcreteModel()
    
    # --- A. Sets & Parameters ---
    flight_ids = df_arrivals['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    # Dictionaries
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x_entry = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y_entry = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_is_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long_arc = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    # Geometry Constants
    r = R_TURN_NM
    x_faf = processed_fixes['VINII']['x']
    y_faf = processed_fixes['VINII']['y']
    
    # --- B. Decision Variables ---
    
    # 1. Path Extension (0 to 20 nm)
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    
    # 2. Arrival Time (seconds)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=lambda m, i: p_tau[i] + 600)
    
    # 3. Variable Speeds (Knots) - With Bounds from your sets
    # We initialize to higher speeds to encourage "fastest possible" flow first
    m.v_L = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(180, 250), initialize=210)
    m.v_theta = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=160)
    m.v_f = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=140)

    # --- C. Geometry Expressions ---
    def calc_distances_expr(m, i):
        y_center_offset = r if p_is_north[i] else -r
        Cx = x_faf - m.d[i]
        Cy = y_faf + y_center_offset
        Cx_prime = x_faf - m.d[i]
        Cy_prime = y_faf
        
        d0_sq = (p_x_entry[i] - Cx)**2 + (p_y_entry[i] - Cy)**2
        d0 = pyo.sqrt(d0_sq + 1e-6)
        d0_prime_sq = (p_x_entry[i] - Cx_prime)**2 + (p_y_entry[i] - Cy_prime)**2
        
        d_L = pyo.sqrt(d0_sq - r**2 + 1e-6)
        
        term1 = r / (d0 + 1e-6)
        term2 = (r**2 + d0_sq - d0_prime_sq) / (2 * r * (d0 + 1e-6))
        theta1 = pyo.acos(term1)
        theta2 = pyo.acos(term2)
        
        if p_long_arc[i]:
            theta_rad = 2*3.14159 - (theta1 + theta2)
        else:
            theta_rad = theta2 - theta1
            
        d_theta = r * theta_rad
        d_final = m.d[i]
        
        return d_L, d_theta, d_final

    # --- D. Constraints ---
    
    # 1. Physics: Time depends on Distance and Variable Speeds
    def physics_rule(m, i):
        d_L, d_theta, d_final = calc_distances_expr(m, i)
        t_L = (d_L / m.v_L[i]) * 3600
        t_turn = (d_theta / m.v_theta[i]) * 3600
        t_final = (d_final / m.v_f[i]) * 3600
        return m.t[i] == p_tau[i] + t_L + t_turn + t_final
    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)
    
    # 2. HARD Separation Constraint (Strict Inequality)
    def separation_rule(m, i):
        prev_i = i - 1
        if prev_i not in m.I:
            return pyo.Constraint.Skip
        # STRICT: Current time must be >= Previous time + 64s
        return m.t[i] >= m.t[prev_i] + T_SEP_SEC
    m.c_separation = pyo.Constraint(m.I, rule=separation_rule)
    
    # 3. Speed Monotonicity Constraints (v_L >= v_theta >= v_f)
    def speed_mono_1(m, i):
        return m.v_L[i] >= m.v_theta[i]
    m.c_speed_1 = pyo.Constraint(m.I, rule=speed_mono_1)

    def speed_mono_2(m, i):
        return m.v_theta[i] >= m.v_f[i]
    m.c_speed_2 = pyo.Constraint(m.I, rule=speed_mono_2)
    
    # --- E. Objective ---
    # Minimize Makespan (Arrival time of the last aircraft)
    last_id = flight_ids[-1]
    m.obj = pyo.Objective(expr=m.t[last_id], sense=pyo.minimize)
    
    # --- F. Solve ---
    solver = SolverFactory('ipopt')
    # Increase iter limit because finding a feasible point with hard constraints is harder
    solver.options['max_iter'] = 10000
    
    if not debug:
        solver.options['print_level'] = 0
        
    results_obj = solver.solve(m, tee=debug)
    
    # Check if optimal
    status = results_obj.solver.termination_condition
    print(f"Solver Status: {status}")

    # --- G. Extract ---
    results = []
    # Even if infeasible, we try to extract values to see where it failed
    try:
        for i in m.I:
            results.append({
                'aircraft_id': i,
                'entry_time': p_tau[i],
                'optimized_d_i': pyo.value(m.d[i]),
                'arrival_time_faf': pyo.value(m.t[i]),
                'v_L': pyo.value(m.v_L[i]),
                'v_theta': pyo.value(m.v_theta[i]),
                'v_f': pyo.value(m.v_f[i])
            })
    except ValueError:
        print("Could not extract values (Solver likely failed completely).")
        return pd.DataFrame()
        
    return pd.DataFrame(results)

# --- Run ---
print("Running Co-Optimization (Hard Constraints)...")
df_hard_results = solve_co_optimization_no_slack(df_arrivals, debug=True)

if not df_hard_results.empty:
    df_final_hard = pd.merge(df_arrivals, df_hard_results, on='aircraft_id')
    # Calculate achieved separation
    df_final_hard['prev_arrival'] = df_final_hard['arrival_time_faf'].shift(1)
    df_final_hard['separation'] = df_final_hard['arrival_time_faf'] - df_final_hard['prev_arrival']
    
    display(df_final_hard[['aircraft_id', 'optimized_d_i', 'v_L', 'v_f', 'arrival_time_faf', 'separation']])
else:
    print("Optimization failed to find a feasible solution.")

Running Co-Optimization (Hard Constraints)...
Ipopt 3.14.19: max_iter=10000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:      225
Number of nonzeros in inequality constraint Jacobian.:      268
Number of nonzeros in Lagrangian Hessian.............:      315

Total number of variables............................:      225
                     variables with only lower bounds:       45
                variables with lower and upper bounds:      180
                     variables with only upper bounds:        0
Total number of

,aircraft_id,optimized_d_i,v_L,v_f,arrival_time_faf,separation
0,1,6.940656e+00,227.204391,148.872088,660.583565,NaN
1,2,2.688419e-01,245.300795,153.723333,949.822501,289.238936
2,3,1.621305e+01,220.037636,141.023342,1025.602924,75.780424
3,4,1.569334e+01,220.919273,141.817903,1102.543476,76.940552
4,5,1.432014e+01,224.345297,145.740081,1180.728316,78.184840
5,6,1.659769e+01,219.211745,140.368003,1259.289874,78.561558
6,7,1.589020e+01,221.079359,142.082299,1340.178887,80.889013
7,8,4.732895e-01,242.721310,153.031812,1423.306028,83.127140
8,9,1.360946e+01,224.758477,146.258885,1498.151631,74.845604
9,10,1.536898e+01,223.752593,144.878790,1573.146092,74.994460


### With Slack Variables

In [9]:
def solve_schedule_optimization_robust(df_arrivals, debug=False):
    m = pyo.ConcreteModel()
    
    # Sets & Parameters (Same as before)
    flight_ids = df_arrivals['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x_entry = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y_entry = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_is_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long_arc = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    r = R_TURN_NM
    x_faf = processed_fixes['VINII']['x']
    y_faf = processed_fixes['VINII']['y']
    
    # --- Decision Variables ---
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=lambda m, i: p_tau[i] + 600)
    m.v_L = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(180, 250), initialize=210)
    m.v_theta = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=160)
    m.v_f = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(130, 200), initialize=140)
    
    # NEW: Slack Variable (Amount of separation violation allowed)
    # We initialize it to 0.0. If the solver uses this, it admits defeat on separation.
    m.slack = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=0.0)

    # --- Expressions ---
    def calc_distances_expr(m, i):
        y_center_offset = r if p_is_north[i] else -r
        Cx = x_faf - m.d[i]
        Cy = y_faf + y_center_offset
        Cx_prime = x_faf - m.d[i]
        Cy_prime = y_faf
        
        d0_sq = (p_x_entry[i] - Cx)**2 + (p_y_entry[i] - Cy)**2
        d0 = pyo.sqrt(d0_sq + 1e-6) # Safety epsilon
        d0_prime_sq = (p_x_entry[i] - Cx_prime)**2 + (p_y_entry[i] - Cy_prime)**2
        
        # Tangent d_L
        d_L = pyo.sqrt(d0_sq - r**2 + 1e-6)
        
        # Angles
        term1 = r / (d0 + 1e-6)
        term2 = (r**2 + d0_sq - d0_prime_sq) / (2 * r * (d0 + 1e-6))
        
        theta1 = pyo.acos(term1)
        theta2 = pyo.acos(term2)
        
        if p_long_arc[i]:
            theta_rad = 2*3.14159 - (theta1 + theta2)
        else:
            theta_rad = theta2 - theta1
            
        d_theta = r * theta_rad
        d_final = m.d[i]
        
        return d_L, d_theta, d_final

    # --- Constraints ---
    def physics_rule(m, i):
        d_L, d_theta, d_final = calc_distances_expr(m, i)
        t_L = (d_L / m.v_L[i]) * 3600
        t_turn = (d_theta / m.v_theta[i]) * 3600
        t_final = (d_final / m.v_f[i]) * 3600
        return m.t[i] == p_tau[i] + t_L + t_turn + t_final
    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)
    
    # NEW: Robust Separation Rule
    def separation_rule(m, i):
        prev_i = i - 1
        if prev_i not in m.I:
            return pyo.Constraint.Skip
        
        # Constraint: t[i] must be at least t[i-1] + 64, 
        # BUT we subtract slack[i]. If slack[i] is positive, we allow a violation.
        return m.t[i] >= m.t[prev_i] + T_SEP_SEC - m.slack[i]
        
    m.c_separation = pyo.Constraint(m.I, rule=separation_rule)
    
    # Speed constraints (same as before)
    m.c_speed_1 = pyo.Constraint(m.I, rule=lambda m, i: m.v_L[i] >= m.v_theta[i])
    m.c_speed_2 = pyo.Constraint(m.I, rule=lambda m, i: m.v_theta[i] >= m.v_f[i])
    
    # --- NEW Objective ---
    # Minimize Makespan + Huge Penalty for violating separation
    # Penalty weight = 1000 (1 second of violation costs as much as 1000 seconds of flight time)
    last_id = flight_ids[-1]
    m.obj = pyo.Objective(expr=m.t[last_id] + 1000 * sum(m.slack[i] for i in m.I), sense=pyo.minimize)
    
    # --- Solve ---
    solver = SolverFactory('ipopt')
    if not debug: solver.options['print_level'] = 0
    
    # Increase max iterations to handle the larger search space
    solver.options['max_iter'] = 10000 
    solver.solve(m, tee=debug)
    
    # --- Extract ---
    results = []
    for i in m.I:
        results.append({
            'aircraft_id': i,
            'd_i': pyo.value(m.d[i]),
            'arrival_time': pyo.value(m.t[i]),
            'actual_separation': T_SEP_SEC-pyo.value(m.slack[i]),
            'v_L': pyo.value(m.v_L[i]),
            'v_theta': pyo.value(m.v_theta[i]),
            'v_f': pyo.value(m.v_f[i])
        })
    return pd.DataFrame(results)

print("Optimizing Schedule...")
df_opt_results = solve_schedule_optimization_robust(df_arrivals, debug=True)
df_final = pd.merge(df_arrivals, df_opt_results, on='aircraft_id')
df_final

Optimizing Schedule...


Ipopt 3.14.19: max_iter=10000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:      225
Number of nonzeros in inequality constraint Jacobian.:      312
Number of nonzeros in Lagrangian Hessian.............:      315

Total number of variables............................:      270
                     variables with only lower bounds:       90
                variables with lower and upper bounds:      180
                     variables with only upper bounds:        0
Total number of equality constraints.................:       

,entry_time,corner,enter_fix_name,x_entry,y_entry,x_faf,y_faf,r_turn,is_north,long_arc,aircraft_id,d_i,arrival_time,actual_separation,v_L,v_theta,v_f
0,84.295182,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,1,6.940656e+00,660.583565,66.0,227.204391,176.058540,148.872088
1,166.813758,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,2,2.688419e-01,949.822501,66.0,245.300795,194.514972,153.723333
2,168.418253,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,3,1.621305e+01,1025.602924,66.0,220.037636,166.586424,141.023342
3,258.758770,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,4,1.569334e+01,1102.543476,66.0,220.919273,167.656910,141.817903
4,389.716874,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,5,1.432014e+01,1180.728316,66.0,224.345297,172.689031,145.740081
5,391.302385,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,6,1.659769e+01,1259.289874,66.0,219.211745,165.636729,140.368003
6,477.332348,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,7,1.589020e+01,1340.178887,66.0,221.079359,168.020503,142.082299
7,622.363538,SouthEast,HUSKY,23.466697,-18.211933,-5.090967,0.001485,6.0,False,1,8,4.732895e-01,1423.306028,66.0,242.721310,191.399066,153.031812
8,732.205749,NorthWest,DALAS,-19.922658,19.106464,-5.090967,0.001485,6.0,True,0,9,1.360946e+01,1498.151631,66.0,224.758477,173.201377,146.258885
9,743.612044,SouthWest,TIROE,-20.976252,-19.663862,-5.090967,0.001485,6.0,False,0,10,1.536898e+01,1573.146092,66.0,223.752593,171.747544,144.878790


In [11]:
# Calculate Delay (Arrival Time - Minimum Possible Time)
# Note: This is simplified; true delay is typically (Actual - Scheduled)
# Here we just look at the timeline.
df_final['transit_time'] = df_final['arrival_time'] - df_final['entry_time']

print("\nOptimization Complete.")
display(df_final[['aircraft_id', 'corner', 'entry_time', 'd_i', 'arrival_time', 'transit_time']])


Optimization Complete.


,aircraft_id,corner,entry_time,d_i,arrival_time,transit_time
0,1,SouthWest,84.295182,6.940656e+00,660.583565,576.288383
1,2,SouthEast,166.813758,2.688419e-01,949.822501,783.008742
2,3,NorthWest,168.418253,1.621305e+01,1025.602924,857.184672
3,4,NorthWest,258.758770,1.569334e+01,1102.543476,843.784706
4,5,SouthWest,389.716874,1.432014e+01,1180.728316,791.011442
5,6,NorthWest,391.302385,1.659769e+01,1259.289874,867.987489
6,7,SouthWest,477.332348,1.589020e+01,1340.178887,862.846539
7,8,SouthEast,622.363538,4.732895e-01,1423.306028,800.942490
8,9,NorthWest,732.205749,1.360946e+01,1498.151631,765.945883
9,10,SouthWest,743.612044,1.536898e+01,1573.146092,829.534048


# Two Stage Optimization
 - To enforce the segment speeds only from speed option sets: 

$\mathcal V_L=\{180,190,200,210,220,230,240,250\} \\
\mathcal V_{\theta}=\{130,140,150,160,170,180,190,200\} \\
\mathcal V_F=\{130,140,150,160,170,180,190,200\} \\$

In [13]:
def solve_two_stage_discrete(df_arrivals, debug=False):
    print("--- STAGE 1: Solving Continuous Relaxation ---")
    # 1. Solve with continuous speeds (using the function we wrote previously)
    # Note: Using the robust version (with slack) ensures we get a solution to round
    df_stage1 = solve_schedule_optimization_robust(df_arrivals, debug=False)
    
    if df_stage1.empty:
        print("Stage 1 failed. Cannot proceed.")
        return pd.DataFrame()
        
    print("--- Rounding Speeds ---")
    # 2. Round speeds to nearest 10
    def round_to_10(x): return 10 * round(x / 10)
    
    # Create a dictionary of FIXED speeds for Stage 2
    fixed_v_L = {row['aircraft_id']: round_to_10(row['v_L']) for _, row in df_stage1.iterrows()}
    fixed_v_theta = {row['aircraft_id']: round_to_10(row['v_theta']) for _, row in df_stage1.iterrows()}
    fixed_v_f = {row['aircraft_id']: round_to_10(row['v_f']) for _, row in df_stage1.iterrows()}
    
    # Note: You can do logic here. E.g., "If rounding up violates speed limit, round down"
    # For now, we assume simple rounding keeps us within [180, 250] approx.
    
    print("--- STAGE 2: Re-optimizing d_i with Fixed Discrete Speeds ---")
    
    # 3. Build Stage 2 Model (Similar to "Fixed Speed" model, but speeds vary per aircraft)
    m = pyo.ConcreteModel()
    m.I = pyo.Set(initialize=df_arrivals['aircraft_id'].tolist())
    
    # Parameters
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    # GEOMETRY CONSTANTS
    r = R_TURN_NM
    x_faf, y_faf = processed_fixes['VINII']['x'], processed_fixes['VINII']['y']
    
    # VARIABLES
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals)
    m.slack = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=0.0) # Keep slack for safety

    # EXPRESSIONS & CONSTRAINTS
    def physics_rule(m, i):
        # --- Geometry Calc (Same as before) ---
        y_off = r if p_north[i] else -r
        Cx, Cy = x_faf - m.d[i], y_faf + y_off
        d0 = pyo.sqrt((p_x[i]-Cx)**2 + (p_y[i]-Cy)**2 + 1e-6)
        d0_p = pyo.sqrt((p_x[i]-(x_faf-m.d[i]))**2 + (p_y[i]-y_faf)**2)
        d_L = pyo.sqrt(d0**2 - r**2 + 1e-6)
        
        theta1 = pyo.acos(r/(d0+1e-6))
        theta2 = pyo.acos((r**2 + d0**2 - d0_p**2)/(2*r*(d0+1e-6)))
        theta = (2*3.14159 - (theta1+theta2)) if p_long[i] else (theta2-theta1)
        
        # --- TIME CALC USING FIXED ROUNDED SPEEDS ---
        # We use the fixed dictionary here!
        v_L_val = fixed_v_L[i]  
        v_theta_val = fixed_v_theta[i]
        v_f_val = fixed_v_f[i]
        
        # If you optimized all 3, round all 3 and put them here
        
        return m.t[i] == p_tau[i] + (d_L/v_L_val)*3600 + ((r*theta)/v_theta_val)*3600 + (m.d[i]/v_f_val)*3600

    m.phys = pyo.Constraint(m.I, rule=physics_rule)
    
    def sep_rule(m, i):
        prev = i-1
        if prev not in m.I: return pyo.Constraint.Skip
        return m.t[i] >= m.t[prev] + 64.0 - m.slack[i]
    m.sep = pyo.Constraint(m.I, rule=sep_rule)
    
    # Objective: Minimize Makespan + Penalty
    last = m.I.last()
    m.obj = pyo.Objective(expr=m.t[last] + 1000*sum(m.slack[i] for i in m.I), sense=pyo.minimize)
    
    # SOLVE
    solver = SolverFactory('ipopt')
    solver.options['print_level'] = 0
    solver.solve(m)
    
    # Extract
    res = []
    for i in m.I:
        res.append({
            'aircraft_id': i,
            'optimized_d_i': pyo.value(m.d[i]),
            'v_L_discrete': fixed_v_L[i], # This is now an integer multiple of 10
            'v_theta_discrete': fixed_v_theta[i],
            'v_f_discrete': fixed_v_f[i],
            'arrival_time': pyo.value(m.t[i]),
            'actual_separation': T_SEP_SEC-pyo.value(m.slack[i])
        })
    return pd.DataFrame(res)

# Run it
df_discrete = solve_two_stage_discrete(df_arrivals)
display(df_discrete)

--- STAGE 1: Solving Continuous Relaxation ---


--- Rounding Speeds ---
--- STAGE 2: Re-optimizing d_i with Fixed Discrete Speeds ---


,aircraft_id,optimized_d_i,v_L_discrete,v_theta_discrete,v_f_discrete,arrival_time,actual_separation
0,1,7.085889e+00,230,180,150,656.004567,66.0
1,2,3.093368e-01,250,190,150,950.793674,66.0
2,3,1.635369e+01,220,170,140,1026.091639,66.0
3,4,1.557535e+01,220,170,140,1103.005428,66.0
4,5,1.442050e+01,220,170,150,1181.559723,66.0
5,6,1.700144e+01,220,170,140,1260.692494,66.0
6,7,1.588227e+01,220,170,140,1343.921935,66.0
7,8,4.250658e-01,240,190,150,1429.296903,66.0
8,9,1.373981e+01,220,170,150,1502.666126,66.0
9,10,1.489630e+01,220,170,140,1576.216007,66.0


## MINLP formulation for intervaled segment speed
 - Introduce the binary variables to select exactly one speed for each segment

In [ ]:
# ==========================================
# 8. MINLP EXACT FORMULATION
#    Variables: d_i (Continuous), Speeds (Discrete Sets)
#    Technique: Binary Variables for Speed Selection
# ==========================================

def solve_exact_discrete_minlp(df_arrivals, debug=False):
    m = pyo.ConcreteModel()
    
    # --- A. Sets & Parameters ---
    flight_ids = df_arrivals['aircraft_id'].tolist()
    m.I = pyo.Set(initialize=flight_ids)
    
    # Dictionaries
    p_tau = df_arrivals.set_index('aircraft_id')['entry_time'].to_dict()
    p_x = df_arrivals.set_index('aircraft_id')['x_entry'].to_dict()
    p_y = df_arrivals.set_index('aircraft_id')['y_entry'].to_dict()
    p_north = df_arrivals.set_index('aircraft_id')['is_north'].to_dict()
    p_long = df_arrivals.set_index('aircraft_id')['long_arc'].to_dict()
    
    r = R_TURN_NM
    x_faf, y_faf = processed_fixes['VINII']['x'], processed_fixes['VINII']['y']
    
    # --- B. DISCRETE SPEED SETS (As defined in your LaTeX) ---
    # We perform a little python trick to make them float values for calculation
    Set_VL = [180.0, 190.0, 200.0, 210.0, 220.0, 230.0, 240.0, 250.0]
    Set_Vtheta = [130.0, 140.0, 150.0, 160.0, 170.0, 180.0, 190.0, 200.0]
    Set_VF = [130.0, 140.0, 150.0, 160.0, 170.0, 180.0, 190.0, 200.0]
    
    m.Set_VL = pyo.Set(initialize=Set_VL)
    m.Set_Vtheta = pyo.Set(initialize=Set_Vtheta)
    m.Set_VF = pyo.Set(initialize=Set_VF)

    # --- C. Decision Variables ---
    
    # 1. Path Extension (Continuous)
    m.d = pyo.Var(m.I, domain=pyo.NonNegativeReals, bounds=(0.0, 20.0), initialize=0.5)
    
    # 2. Arrival Time (Continuous)
    m.t = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=lambda m, i: p_tau[i] + 600)
    
    # 3. Slack (Continuous) - Essential for feasibility with discrete sets
    m.slack = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=0.0)

    # 4. BINARY VARIABLES for Speed Selection
    # z[i, v] = 1 if aircraft i selects speed v, 0 otherwise
    m.z_L = pyo.Var(m.I, m.Set_VL, domain=pyo.Binary)
    m.z_theta = pyo.Var(m.I, m.Set_Vtheta, domain=pyo.Binary)
    m.z_F = pyo.Var(m.I, m.Set_VF, domain=pyo.Binary)

    # --- D. Speed Constraints (The "Exact" Part) ---

    # 1. Select Exactly One Speed per Segment
    def select_one_L(m, i):
        return sum(m.z_L[i, v] for v in m.Set_VL) == 1
    m.c_sel_L = pyo.Constraint(m.I, rule=select_one_L)

    def select_one_theta(m, i):
        return sum(m.z_theta[i, v] for v in m.Set_Vtheta) == 1
    m.c_sel_theta = pyo.Constraint(m.I, rule=select_one_theta)

    def select_one_F(m, i):
        return sum(m.z_F[i, v] for v in m.Set_VF) == 1
    m.c_sel_F = pyo.Constraint(m.I, rule=select_one_F)

    # 2. Define Effective Speed (Helper Expressions)
    # This calculates the actual scalar speed based on the binary selection
    def get_vL(m, i):
        return sum(v * m.z_L[i, v] for v in m.Set_VL)
    
    def get_vTheta(m, i):
        return sum(v * m.z_theta[i, v] for v in m.Set_Vtheta)
    
    def get_vF(m, i):
        return sum(v * m.z_F[i, v] for v in m.Set_VF)

    # 3. Monotonicity: v_L >= v_theta >= v_F
    # We can enforce this directly on the expressions
    def speed_mono_1(m, i):
        return get_vL(m, i) >= get_vTheta(m, i)
    m.c_mono_1 = pyo.Constraint(m.I, rule=speed_mono_1)

    def speed_mono_2(m, i):
        return get_vTheta(m, i) >= get_vF(m, i)
    m.c_mono_2 = pyo.Constraint(m.I, rule=speed_mono_2)

    # --- E. Physics & Geometry ---
    def physics_rule(m, i):
        # Geometry (Same as previous setups)
        y_off = r if p_north[i] else -r
        Cx, Cy = x_faf - m.d[i], y_faf + y_off
        d0 = pyo.sqrt((p_x[i]-Cx)**2 + (p_y[i]-Cy)**2 + 1e-6)
        d0_p = pyo.sqrt((p_x[i]-(x_faf-m.d[i]))**2 + (p_y[i]-y_faf)**2)
        d_L = pyo.sqrt(d0**2 - r**2 + 1e-6)
        
        theta1 = pyo.acos(r/(d0+1e-6))
        theta2 = pyo.acos((r**2 + d0**2 - d0_p**2)/(2*r*(d0+1e-6)))
        theta_rad = (2*3.14159 - (theta1+theta2)) if p_long[i] else (theta2-theta1)
        d_theta = r * theta_rad
        d_final = m.d[i]
        
        # TIME CALCULATION: Using the Sum-Product of Binaries
        # Note: Division by variable (speed) makes this Non-Linear
        t_L = (d_L / get_vL(m, i)) * 3600
        t_turn = (d_theta / get_vTheta(m, i)) * 3600
        t_final = (d_final / get_vF(m, i)) * 3600
        
        return m.t[i] == p_tau[i] + t_L + t_turn + t_final

    m.c_physics = pyo.Constraint(m.I, rule=physics_rule)

    # --- F. Separation ---
    def separation_rule(m, i):
        prev_i = i - 1
        if prev_i not in m.I: return pyo.Constraint.Skip
        return m.t[i] >= m.t[prev_i] + T_SEP_SEC - m.slack[i]
    m.c_separation = pyo.Constraint(m.I, rule=separation_rule)

    # --- G. Objective ---
    last_id = flight_ids[-1]
    m.obj = pyo.Objective(expr=m.t[last_id] + 1000 * sum(m.slack[i] for i in m.I), sense=pyo.minimize)

    # --- H. SOLVE using MindtPy ---
    # Strategy: Outer Approximation (OA) decomposes the problem
    # It uses 'ipopt' for the NLP part and 'glpk' (or 'cbc') for the Integer part
    print("Solving MINLP with MindtPy...")
    
    # Ensure you have 'glpk' or 'cbc' installed for the MIP part!
    solver = SolverFactory('mindtpy')
    
    try:
        # mip_solver='glpk' is standard, can change to 'cbc' or 'gurobi'
        # nlp_solver='ipopt' handles the geometry
        results = solver.solve(m, mip_solver='glpk', nlp_solver='ipopt', tee=debug, 
                               iteration_limit=20) 
    except Exception as e:
        print(f"Solver Failed: {e}")
        print("Ensure you have 'glpk' installed (conda install glpk)")
        return pd.DataFrame()

    # --- Extract ---
    res = []
    for i in m.I:
        # We must iterate binaries to find which one was selected
        sel_vL = sum(v for v in m.Set_VL if pyo.value(m.z_L[i, v]) > 0.5)
        sel_vTheta = sum(v for v in m.Set_Vtheta if pyo.value(m.z_theta[i, v]) > 0.5)
        sel_vF = sum(v for v in m.Set_VF if pyo.value(m.z_F[i, v]) > 0.5)
        
        res.append({
            'aircraft_id': i,
            'optimized_d_i': pyo.value(m.d[i]),
            'v_L': sel_vL,
            'v_theta': sel_vTheta,
            'v_f': sel_vF,
            'arrival_time': pyo.value(m.t[i]),
            'slack': pyo.value(m.slack[i])
        })
    return pd.DataFrame(res)

# --- Instructions to Run ---
# Uncomment the line below ONLY if you have 'glpk' installed alongside 'ipopt'
df_minlp = solve_exact_discrete_minlp(df_arrivals, debug=True)
display(df_minlp)

Starting MindtPy version 1.0.0 using OA algorithm
iteration_limit: 20
stalling_limit: 15
time_limit: 600
strategy: OA
add_regularization: None
call_after_main_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x7db6c248f2e0>
call_before_subproblem_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x7db6c248f310>
call_after_subproblem_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x7db6c248f340>
call_after_subproblem_feasible: <pyomo.contrib.gdpopt.util._DoNothing object at 0x7db6c248f370>
tee: true
logger: <Logger pyomo.contrib.mindtpy (INFO)>
logging_level: 20
integer_to_binary: false
add_no_good_cuts: false
use_tabu_list: false
single_tree: false
solution_pool: false
num_solution_iteration: 5
cycling_check: true
feasibility_norm: L_infinity
differentiate_mode: reverse_symbolic
use_mcpp: false
calculate_dual_at_solution: false
use_fbbt: false
use_dual_bound: true
partition_obj_nonlinear_terms: true
quadratic_strategy: 0
move_objective: false
add_cuts_at_incumbent: f

Solving MINLP with MindtPy...
deprecated. Either specify deactivated Blocks as targets to activate them if
transforming them is the desired behavior.  (deprecated in 6.9.3) (called from
/home/yp6443/miniconda3/envs/tracon/lib/python3.10/site-
packages/pyomo/core/base/transformation.py:77)


         -       Relaxed NLP           4165.56            inf        4165.56      nan%      0.36
         1              MILP           4165.56            inf        4165.56      nan%      0.55
*        1         Fixed NLP           74779.8        74779.8        4165.56    94.43%      0.80


# Setup III: Co-optimize $d_i$, segments speed, and the arrival sequence (Do Not Follow FCFS Rule)